# Assignment 5

Deadline: 11.06.2025 12:00 CEST

## Task

Develop an investment strategy for the Swiss equity market, backtest it using the provided datasets (`market_data.parquet`, `jkp_data.parquet`, `spi_index.csv`) and analyze its performance by benchmarking it against the SPI index. Work with the existing code infrastructure (`qpmwp-course`) and extend it by implementing any additional components needed for the strategy. Write a report that presents your methodology and the results.

### Coding (15 points)

- Selection:
  Implement selection item builder functions (via `SelectionItemBuilder`) to filter stocks based on specific criteria (e.g., exclude low-quality or high-volatility stocks).

- Optimization Data & Constraints:
  Implement functions to prepare optimization data (via `OptimizationItemBuilder`), including any econometric or machine learning-based predictions. These functions should also define optimization constraints (e.g., stock, sector, or factor exposure limits).

- Optimization Model:
  If you choose to create a custom optimization model, develop a class inheriting from Optimization (similar to `MeanVariance`, `LeastSquares`, or `BlackLitterman`). Your class should include methods set_objective and solve for defining the objective function and solving the optimization problem.

- Machine Learning Prediction:
  Integrate a machine learning model to estimate inputs for the optimization, such as expected returns or risk. This could include regression, classification, or learning-to-rank models. I suggest you to use the provided jkp_data as features, but you may also create your own (e.g., technical indicators computed on the return or price series).

- Simulation:
  Backtest the strategy and simulate portfolio returns. Account for fixed costs (1% per annum) and variable (transaction) costs (0.2% per rebalancing).


### Report (15 points):

Generate an HTML report with the following sections:

- High-level strategy overview: Describe the investment strategy you developed.

- Detailed explanation of the backtesting steps: Offer a more comprehensive breakdown of the backtesting process, including a description of the models implemented (e.g., details of the machine learning method used).

- Backtesting results:
    
    - Charts: Include visual representations (e.g., cumulative performance charts, rolling 3-year returns, etc.).
    - Descriptive statistics: Present key statistics such as mean, standard deviation, drawdown, turnover, and Sharpe ratio (or any other relevant metric) for the full backtest period as well as for subperiods (e.g., the last 5 years, or during bull vs. bear market phases).
    - Compare your strategy against the SPI index.


In [ ]:
# Standard library imports
import os
import sys
import types

# Third party imports
import numpy as np
import pandas as pd

# Add the project root directory to Python path
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()
src_path = os.getenv('PROJECT_SOURCE_DIR')
#print(src_path)
sys.path.append(src_path)

# Local modules imports
from helper_functions import load_data_spi, load_pickle
from estimation.covariance import Covariance
from estimation.black_litterman import (
    bl_posterior_mean,                              
    generate_views_from_scores,                     
)
from optimization.optimization import (
    BlackLitterman,                                 
)
from backtesting.backtest_item_builder_classes import (
    SelectionItemBuilder,
    OptimizationItemBuilder,
)
from backtesting.backtest_item_builder_functions import (
    # Selection item builder functions
    bibfn_selection_min_volume,
    bibfn_selection_NA,
    bibfn_selection_gaps,
    bibfn_selection_jkp_factor_scores,

    # Optimization item builder functions
    bibfn_return_series,
    bibfn_bm_series,
    bibfn_cap_weights, 
    bibfn_size_dependent_upper_bounds,
    bibfn_scores,
    bibfn_turnover_constraint,        
                            
    # Constraints item builder functions
    bibfn_budget_constraint,
    bibfn_box_constraints,
)
from backtesting.backtest_data import BacktestData
from backtesting.backtest_service import BacktestService
from backtesting.backtest import Backtest


In [2]:
# Load data
path_to_data = '../data/'  # <change this to your path to data>

# Load market and jkp data from parquet files
market_data = pd.read_parquet(path = f'{path_to_data}market_data.parquet')
jkp_data = pd.read_parquet(path = f'{path_to_data}jkp_data.parquet')

# Instantiate the BacktestData class
# and set the market data and jkp data as attributes
data = BacktestData()
data.market_data = market_data
data.jkp_data = jkp_data
data.bm_series = load_data_spi(path='../data/')

# Define rebalancing dates
## ToDo Rebalancing 
n_days = 21*3
market_data_dates = market_data.index.get_level_values('date').unique().sort_values(ascending=True)
rebdates = market_data_dates[market_data_dates > '2005-01-01'][::n_days].strftime('%Y-%m-%d').tolist()

In [4]:
JKP_FIELDS = ['qmj', 'ret_12_1', 'resff3_6_1']

selection_item_builders = {
    # Liquidity
    'min_volume': SelectionItemBuilder(
        bibfn=bibfn_selection_min_volume,
        width=252,
        min_volume=100_000,
        agg_fn=np.median,
    ),
    
    # Trading continuity
    'gaps': SelectionItemBuilder(
        bibfn=bibfn_selection_gaps,
        width=252 * 3,
        n_days=10,
    ),

    # Data quality
    'NA_check': SelectionItemBuilder(
        bibfn=bibfn_selection_NA,
        width=252,
        na_threshold=10,
    ),

    'scores': SelectionItemBuilder(
        bibfn=bibfn_selection_jkp_factor_scores,
        fields= JKP_FIELDS,  # your custom list
    ),
}

optimization_item_builders = {
    # 1. Return series for optimization objective
    'return_series': OptimizationItemBuilder(
        bibfn=bibfn_return_series,
        width=252 * 3,
    ),

    # 2. Benchmark return series (e.g., SPI), useful for relative return / tracking error
    'bm_series': OptimizationItemBuilder(
        bibfn=bibfn_bm_series,
        width=252 * 3,
        align=True,
        name='bm_series',
    ),

    # 3. Budget constraint: fully invested portfolio
    'budget_constraint': OptimizationItemBuilder(
        bibfn=bibfn_budget_constraint,
        budget=1,
    ),

    # 4. Box constraints: e.g., long-only with max weight per stock
    'box_constraints': OptimizationItemBuilder(
        bibfn=bibfn_box_constraints,
        lower=0.0,
        upper=0.2,
        box_type='LongOnly',
    ),

    # 5. Size-dependent upper bounds by market cap (small, mid, large caps)
    'size_dep_upper_bounds': OptimizationItemBuilder(
        bibfn=bibfn_size_dependent_upper_bounds,
        small_cap={'threshold': 300_000_000, 'upper': 0.02},
        mid_cap={'threshold': 1_000_000_000, 'upper': 0.05},
        large_cap={'threshold': 10_000_000_000, 'upper': 0.1},
    ),

    # 6. Cap-weighted benchmark weights (for optimization objectives or constraints)
    'cap_weights': OptimizationItemBuilder(
        bibfn=bibfn_cap_weights,
    ),

    # 7. Optional: Turnover constraint to manage transaction costs
    'turnover_constraint': OptimizationItemBuilder(
        bibfn=bibfn_turnover_constraint,
        turnover_limit=0.2,  # Adjust based on your tolerance
    ),

    'factor_scores': OptimizationItemBuilder(
    bibfn=bibfn_scores,  # this copies the scores from the selection into optimization_data
),
    
}

optimization = BlackLitterman(
    solver_name='cvxopt',
    covariance=Covariance(method='ledoit_wolf'),
    risk_aversion=1,
    tau_psi=0.01,
    tau_omega=0.0001,
    view_method='absolute',
    scalefactor=1,
    fields=JKP_FIELDS,  # injects your factor-based views
)

# Initialize the backtest service
bs = BacktestService(
    data=data,
    optimization=optimization,
    selection_item_builders=selection_item_builders,
    optimization_item_builders=optimization_item_builders,
    rebdates=rebdates,
)

# Run the backtest
bs.optimization.params['fields'] = JKP_FIELDS
bt_bl_mom = Backtest()
bt_bl_mom.run(bs=bs)

Rebalancing date: 2005-01-03


RuntimeError: Estimation method not recognized.